In [1]:
from agent.latent_space.LFSS import PretrainedAEModel
import torch

g:\Code\Python\GitHub\Portfolio-Optimizer\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda')

In [3]:
ae = PretrainedAEModel(device=device)

In [4]:
test = torch.randn([3, 8, 60], dtype=torch.float32)

In [6]:
output = ae.predict(test)